# Recommendations On Evaluation Set
In this notebook, we test 100 different user prompts and generated 5 outfits using each prompt. We will evaluate these recommendations in AWS sagemaker ground truth by asking mechanical turk workers to label the recommended outfits as either good or bad.

In [ ]:
%%capture
# Install required packages
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate
!pip install torch
!pip install pandas
!pip install numpy
!pip install huggingface_hub[hf_transfer]

In [ ]:
import pandas as pd
import numpy as np

import os
import random
from PIL import Image as PIL_Image
import tempfile
import io
from pathlib import Path
from IPython.display import Image
from PIL import ImageEnhance
import json
from typing import Union, Dict, List
import re

import torch
from huggingface_hub import hf_hub_download, snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

## Create system prompt to use for recommendation
Take the features described in the recognition pipeline to create a catalog of the user's wardrobe to feed into the recommendation LLM.

In [ ]:
def create_detailed_item_description(item: dict, item_id: str) -> str:
    """Create a detailed description of a clothing item using all available information."""
    description_parts = []

    # Start with the ID reference
    description_parts.append(f"[Item {item_id}]")

    # Basic item description
    base_desc = f"{item['Primary Color']} {item['Type of clothing']}"
    if item['Pattern'] != "Solid" and item['Pattern'] != "Not applicable":
        base_desc = f"{item['Pattern']} {base_desc}"
    description_parts.append(base_desc)

    # Rest of the description logic remains the same...
    if item['Style'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"{item['Style']} style")

    if item['Fabric Material'] not in ["Not specified", "Not applicable"]:
        material_desc = item['Fabric Material']
        if item['Fabric Weight'] not in ["Not specified", "Not applicable"]:
            material_desc = f"{item['Fabric Weight']} {material_desc}"
        description_parts.append(material_desc)

    if item['Fit'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"{item['Fit']} fit")
    if item['Neckline'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"{item['Neckline']} neckline")

    if item['Accent Color(s)'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"with {item['Accent Color(s)']} accents")

    if item['Key design elements'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"featuring {item['Key design elements']}")

    if item['Functional Features'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"({item['Functional Features']})")

    if item['Brand'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"from {item['Brand']}")

    if item['Shape'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"{item['Shape']} shape")

    if item['Occasion suitability'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"suitable for {item['Occasion suitability']}")

    if item['Weather appropriateness'] not in ["Not specified", "Not applicable"]:
        description_parts.append(f"suitable for {item['Weather appropriateness']} weather")

    full_description = ", ".join(description_parts)

    extra_info = []
    # if item['How it can be styled'] not in ["Not specified", "Not applicable"]:
    #     extra_info.append(f"Styling suggestions: {item['How it can be styled']}")
    if item['Additional Notes'] not in ["Not specified", "Not applicable"]:
        extra_info.append(f"Note: {item['Additional Notes']}")

    if extra_info:
        full_description += "\n    " + "\n    ".join(extra_info)

    return full_description

def create_outfit_recommendation_prompt(clothing_items: List[str]) -> tuple[str, List[dict]]:
    # Parse each JSON string into a dictionary and create ID mapping
    items = []

    for idx, item_str in enumerate(clothing_items):
        try:
          item = json.loads(item_str)
          item_id = f"{idx}"
          items.append(item)
        except:
          #print(item_str)
          continue


    # Organize by category
    inventory_by_category = {}

    for idx, item in enumerate(items):
        category = item["Category"]
        if category not in inventory_by_category:
            inventory_by_category[category] = []

        item_id = f"{idx}"
        inventory_by_category[category].append(
            create_detailed_item_description(item, item_id)
        )

    # Create the inventory section
    inventory_text = ""
    for category, items in inventory_by_category.items():
        inventory_text += f"\n{category}:\n"
        for item in items:
            inventory_text += f"- {item}\n"

    # Create the final prompt
    prompt = f"""As a fashion stylist, please create outfit combinations using the following detailed inventory of clothing items. Each item is marked with a unique identifier [Item ###] - please use these identifiers when referring to specific items in your recommendations.

AVAILABLE INVENTORY:
{inventory_text}

INSTRUCTIONS:
1. Only recommend items that are explicitly listed in the inventory above
2. An outfit should always have 1 top, 1 bottom, and 1 footwear item. There can be 0 to 2 outerwear items depending on the occasion and weather.
3. Pay attention to:
   - Weather appropriateness for the requested situation
   - Occasion suitability
   - Color coordination and user color preferences if specified
   - Style consistency with the user's request
4. Provide recommendations in this exact JSON format:
{{
    "Outfit": [1, 2, ...],  // Array of item IDs used in the outfit
    "Explanation": 1-2 concise sentences explaining why these pieces work well together for the requested purpose,
    "Styling Tips": 1-2 concise sentences with styling tips or suggested accessories
}}

FORMAT REQUIREMENTS:
- "Outfit": Array of ID numbers only (without "Item" prefix)
- "Explanation": 1-2 sentences maximum
- "Styling Tips": 1-2 sentences maximum
- Response must be valid JSON
- Only include items from the inventory

The user will provide specific outfit requests. Respond with appropriate outfit combinations that match their request while following the format above."""


    return prompt

In [ ]:
def clean_json_string(json_string):
    # Find the start and end of the valid JSON string
    start_index = json_string.find("{")
    end_index = json_string.rfind("}") + 1
    valid_json_string = json_string[start_index:end_index]
    return valid_json_string

In [ ]:
def filter_image_paths(df):
    """
    Filters a dataframe to keep only rows where the image_path column
    ends with either:
    1. A slash (/) followed by purely numeric values and .jpg
    2. A slash (/) followed by "hm_fashion_images_image_" then numeric values and .jpg

    Args:
        df: Pandas DataFrame with an 'image_path' column

    Returns:
        Filtered DataFrame
    """
    # Define the regex pattern to match both cases:
    # 1. /digits.jpg at the end
    # 2. /hm_fashion_images_image_digits.jpg at the end
    pattern = r'\/(hm_fashion_images_image_)?[0-9]+\.jpg$'

    # Create a boolean mask where True means the path matches our pattern
    mask = df['image_path'].str.contains(pattern, regex=True)

    # Filter the dataframe using the mask
    filtered_df = df[mask]

    # Return the filtered dataframe
    return filtered_df

my_wardrobe = filter_image_paths(pd.read_csv("/content/drive/MyDrive/MIDS Capstone/eval_set.csv"))
#my_wardrobe.drop(columns=["Unnamed: 0"], inplace=True)
my_wardrobe

<ipython-input-5-6315c31932d5>:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['image_path'].str.contains(pattern, regex=True)


,image_path,json_string,Category,Type of clothing,Primary Color,Accent Color(s),Pattern,Shape,Fit,Neckline,Key design elements,Brand,Style,Occasion suitability,Weather appropriateness,Fabric Material,Fabric Weight,Functional Features,Additional Notes
1,/content/final_dataset/12991.jpg,"```json\n{\n ""Category"": ""Footwear"",\n ""Type...",Footwear,Running Shoe,White,"Blue, Gray",Striped,Standard,Standard,Not applicable,"Lace-up closure, mesh upper, cushioned sole, b...",Adidas,Athleisure,"Casual, Sporty",All-season,Synthetic,Medium,"Breathable, Cushioned",The shoe combines functionality with a sporty ...
3,/content/final_dataset/49498.jpg,"```json\n{\n ""Category"": ""Footwear"",\n ""Type...",Footwear,Sneakers,Black,White,Solid,Classic low-top,Standard,Not applicable,"Lace-up closure, Vans logo on tongue, wingtip ...",Vans,Streetwear,Casual,All-season,Canvas,Medium,Not applicable,The Vans Era sneakers feature a classic design...
6,/content/final_dataset/31218.jpg,"```json\n{\n ""Category"": ""Footwear"",\n ""Type...",Footwear,Sneaker,Gray,Blue,Solid,Low-top,Standard,Not applicable,"Lace-up closure, logo on tongue, white sole",DC Shoes,Streetwear,Casual,All-season,Canvas,Medium,Not specified,The sneaker features a sleek design with a com...
9,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Outerwear"",\n ""Typ...",Outerwear,Blazer,Light Beige,Gold,Solid,Structured,Slim-Fit,Not applicable,"Double-breasted, belt with tie, gold buttons",H&M,Minimalist,"Casual, Formal",All-Season,Wool Blend,Medium,Not applicable,This blazer combines elegance with a modern tw...
12,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Top"",\n ""Type of c...",Top,Dress,Blue,White,Embroidered lace,A-line,Relaxed,Round,"Short sleeves with scalloped edges, embroidere...",H&M,Minimalist,Casual,All-season,Cotton,Lightweight,Not applicable,This dress combines a simple silhouette with d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2533,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Top"",\n ""Type of c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2534,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Bottom"",\n ""Type o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2535,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Bottom"",\n ""Type o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2538,/content/final_dataset/hm_fashion_images_image...,"```json\n{\n ""Category"": ""Top"",\n ""Type of c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
my_wardrobe_subset = my_wardrobe.sample(500)
clothing_items = [clean_json_string(s) for s in my_wardrobe_subset["json_string"]]
sys_prompt = create_outfit_recommendation_prompt(clothing_items)
sys_prompt

'As a fashion stylist, please create outfit combinations using the following detailed inventory of clothing items. Each item is marked with a unique identifier [Item ###] - please use these identifiers when referring to specific items in your recommendations.\n\nAVAILABLE INVENTORY:\n\nTop:\n- [Item 0], Vertical Stripes White Button-Up Shirt, Classic/Traditional style, Medium Cotton, Slim-Fit fit, with Blue accents, featuring Buttons, Collar, Standard shape, suitable for Formal, suitable for All-Season weather\n    Note: The shirt features a classic vertical stripe pattern with a slim fit, making it suitable for formal occasions. The white base color with blue stripes adds a touch of sophistication and versatility.\n- [Item 2], Embroidered Cutwork Mint Green Short-Sleeve Henley Shirt, Minimalist style, Lightweight Cotton, Relaxed fit, Henley neckline, with Not Applicable accents, featuring Embroidered cutwork at the chest, button details, short sleeves, (Not Applicable), from Not Speci

## Run Recommendation using Distilled Deepseek Model

In [ ]:
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

# Define the model repository ID
repo_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# Download the model file using hf_transfer
file_path = snapshot_download(repo_id)

print(f"Model downloaded to: {file_path}")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

...

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/6a6f4aa4197940add57724a7707d069478df56b1


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B", torch_dtype="auto", device_map="auto"
)

processor = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def rec_pipeline(df, sys_prompt, user_prompt):
  # run model
  response = run_one_recommendation(sys_prompt, user_prompt)

  # clean output
  rec_output = parse_rec_response(response)

  # display recommended items images, explanation, and styling tips
  #display_recommended_items(df, rec_output)

  # append the prompt and image paths to output dict
  rec_output["prompt"] = user_prompt
  image_paths = []
  for i in rec_output["outfit"]:
    item_info = df.iloc[i]
    image_path = item_info["image_path"]
    image_paths.append(image_path)
  rec_output["image_paths"] = image_paths

  return rec_output


def run_one_recommendation(sys_prompt, user_prompt):
  messages = [
      {"role": "system", "content": sys_prompt},
      {"role": "user", "content": user_prompt}
  ]
  text = processor.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )

  inputs = processor(
      text=[text],
      padding=True,
      return_tensors="pt",
  )

  inputs = inputs.to("cuda")
  model.generation_config.pad_token_id = model.generation_config.eos_token_id


  # Inference: Generation of the output
  generated_ids = model.generate(**inputs, max_new_tokens=1024)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = processor.batch_decode(
      generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  return output_text[0]


def parse_rec_response(response):
    # Define regex patterns to extract each component
    outfit_pattern = r"\"Outfit\":\s*(.*)"
    explanation_pattern = r"\"Explanation\":\s*(.*)"
    styling_pattern = r"\"Styling Tips\":\s*(.*)"

    # Extract the components using regex
    outfit_match = re.search(outfit_pattern, response)
    explanation_match = re.search(explanation_pattern, response)
    styling_match = re.search(styling_pattern, response)

    # Check if any match is None and handle it gracefully
    if not outfit_match or not explanation_match or not styling_match:
        raise ValueError("The response format is invalid or does not match the expected pattern.")

    outfit = outfit_match.group(1).strip()
    outfit = outfit.replace("[", "").replace("]", "")[:-1] # remove [] chars and empty str at end index
    outfit = [int(item) for item in outfit.split(',')]
    explanation = explanation_match.group(1).strip()
    styling = styling_match.group(1).strip()

    # Create and return the dictionary
    return {
        "outfit": outfit,
        "explanation": explanation,
        "styling": styling
    }


def display_recommended_items(df, rec_output):
    for i in rec_output["outfit"]:
        item_info = df.iloc[i]
        image_path = item_info["image_path"]
        temp_path = reduce_image_size(image_path)
        display(Image(filename=temp_path))
        os.remove(temp_path)
    print(f"Explanation: {rec_output['explanation']}")
    print(f"Styling Tips: {rec_output['styling']}")


def reduce_image_size(image_path, scale=16, interpolation=PIL_Image.LANCZOS):
    img = PIL_Image.open(image_path)

    enhancer = ImageEnhance.Contrast(img)
    contrast_enhanced = enhancer.enhance(1.5) # Increase contrast by 50%
    enhancer = ImageEnhance.Color(contrast_enhanced)
    color_enhanced = enhancer.enhance(1.3) # Increase saturation by 30%

    # Convert to RGB if necessary
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Get original dimensions
    width, height = img.size

    # Calculate new dimensions
    new_width = width // scale
    new_height = height // scale

    # Resize the image using the specified interpolation method
    resized_img = color_enhanced.resize((new_width, new_height), interpolation)

    # Create a temporary file path
    temp_path = os.path.join(tempfile.gettempdir(), 'resized_image.png')

    # Save the resized image with original color profile and no compression
    resized_img.save(temp_path)

    return temp_path

In [ ]:
# List of gendered prompts organized by category
gendered_prompts = {
    "Occasion/Activity Prompts": [
        {"male": "Men's outfit for a formal wedding", "female": "Women's outfit for a formal wedding"},
        {"male": "Men's work attire for a corporate office environment", "female": "Women's work attire for a corporate office environment"},
        {"male": "Men's casual outfit for weekend brunch with friends", "female": "Women's casual outfit for weekend brunch with friends"},
        {"male": "Men's athletic wear for a morning jog in the park", "female": "Women's athletic wear for a morning jog in the park"},
        {"male": "Men's business casual outfit for a job interview", "female": "Women's business casual outfit for a job interview"},
        {"male": "Men's outfit for a first date at an upscale restaurant", "female": "Women's outfit for a first date at an upscale restaurant"},
        {"male": "Men's attire for a weekend music festival", "female": "Women's attire for a weekend music festival"},
        {"male": "Men's professional outfit for delivering a conference presentation", "female": "Women's professional outfit for delivering a conference presentation"},
        {"male": "Men's casual look for a quick grocery shopping trip", "female": "Women's casual look for a quick grocery shopping trip"},
        {"male": "Men's comfortable outfit for a family reunion picnic", "female": "Women's comfortable outfit for a family reunion picnic"},
        {"male": "Men's cocktail attire for an evening charity gala", "female": "Women's cocktail attire for an evening charity gala"},
        {"male": "Men's workout clothes for a yoga class", "female": "Women's workout clothes for a yoga class"},
        {"male": "Men's outfit for attending a graduation ceremony", "female": "Women's outfit for attending a graduation ceremony"},
        {"male": "Men's casual look for studying at a coffee shop", "female": "Women's casual look for studying at a coffee shop"},
        {"male": "Men's attire for a night out at a trendy nightclub", "female": "Women's attire for a night out at a trendy nightclub"},
        {"male": "Men's smart casual outfit for a dinner party at a friend's home", "female": "Women's smart casual outfit for a dinner party at a friend's home"},
        {"male": "Men's hiking outfit for a day on the trails", "female": "Women's hiking outfit for a day on the trails"},
        {"male": "Men's business formal attire for an important board meeting", "female": "Women's business formal attire for an important board meeting"},
        {"male": "Men's comfortable outfit for a movie night with friends", "female": "Women's comfortable outfit for a movie night with friends"},
        {"male": "Men's respectful attire for attending a religious service", "female": "Women's respectful attire for attending a religious service"}
    ],
    "Style/Color Scheme Prompts": [
        {"male": "Men's minimalist outfit featuring neutral tones", "female": "Women's minimalist outfit featuring neutral tones"},
        {"male": "Men's vintage-inspired look with earth tone color palette", "female": "Women's vintage-inspired look with earth tone color palette"},
        {"male": "Men's bold outfit incorporating primary colors", "female": "Women's bold outfit incorporating primary colors"},
        {"male": "Men's sophisticated monochromatic black outfit", "female": "Women's sophisticated monochromatic black outfit"},
        {"male": "Men's preppy style outfit with pastel color accents", "female": "Women's preppy style outfit with pastel color accents"},
        {"male": "Men's streetwear look with urban aesthetic elements", "female": "Women's streetwear look with urban aesthetic elements"},
        {"male": "Men's bohemian outfit with layered textures and patterns", "female": "Women's bohemian outfit with layered textures and patterns"},
        {"male": "Men's professional outfit in navy and gray color scheme", "female": "Women's professional outfit in navy and gray color scheme"},
        {"male": "Men's retro 90s inspired outfit with period-appropriate details", "female": "Women's retro 90s inspired outfit with period-appropriate details"},
        {"male": "Men's all-white outfit for summer events", "female": "Women's all-white outfit for summer events"},
        {"male": "Men's edgy outfit combining black with red accent pieces", "female": "Women's edgy outfit combining black with red accent pieces"},
        {"male": "Men's classic look built around a blue and white color scheme", "female": "Women's classic look built around a blue and white color scheme"},
        {"male": "Men's athleisure outfit incorporating bold prints", "female": "Women's athleisure outfit incorporating bold prints"},
        {"male": "Men's romantic date night outfit with soft pink accents", "female": "Women's romantic date night outfit with soft pink accents"},
        {"male": "Men's outfit featuring complementary green and purple colors", "female": "Women's outfit featuring complementary green and purple colors"},
        {"male": "Men's country-western inspired outfit with authentic details", "female": "Women's country-western inspired outfit with authentic details"},
        {"male": "Men's nautical style outfit with navy and white striped elements", "female": "Women's nautical style outfit with navy and white striped elements"},
        {"male": "Men's sophisticated outfit showcasing rich jewel tones", "female": "Women's sophisticated outfit showcasing rich jewel tones"},
        {"male": "Men's sporty outfit with bright accent colors", "female": "Women's sporty outfit with bright accent colors"},
        {"male": "Men's elegant outfit incorporating subtle metallic details", "female": "Women's elegant outfit incorporating subtle metallic details"}
    ],
    "Weather Condition Prompts": [
        {"male": "Men's outfit for staying cool on a hot and humid summer day", "female": "Women's outfit for staying cool on a hot and humid summer day"},
        {"male": "Men's winter outfit for protection in below freezing temperatures", "female": "Women's winter outfit for protection in below freezing temperatures"},
        {"male": "Men's practical outfit for a rainy day", "female": "Women's practical outfit for a rainy day"},
        {"male": "Men's light layered outfit for unpredictable spring weather", "female": "Women's light layered outfit for unpredictable spring weather"},
        {"male": "Men's outfit for a cool autumn day with light breeze", "female": "Women's outfit for a cool autumn day with light breeze"},
        {"male": "Men's summer evening outfit for when temperatures cool down", "female": "Women's summer evening outfit for when temperatures cool down"},
        {"male": "Men's outfit for protection during a heavy snowfall", "female": "Women's outfit for protection during a heavy snowfall"},
        {"male": "Men's lightweight outfit for comfort in a dry, hot desert climate", "female": "Women's lightweight outfit for comfort in a dry, hot desert climate"},
        {"male": "Men's outfit for a foggy, damp morning commute", "female": "Women's outfit for a foggy, damp morning commute"},
        {"male": "Men's adaptable layered outfit for rapidly changing temperatures", "female": "Women's adaptable layered outfit for rapidly changing temperatures"}
    ]
}

# If you need a flat list of all prompts:
all_prompts = []

for category in gendered_prompts:
    for prompt_pair in gendered_prompts[category]:
        all_prompts.append(prompt_pair["male"])
        all_prompts.append(prompt_pair["female"])

# Example usage:
all_prompts[0:5]

["Men's outfit for a formal wedding",
 "Women's outfit for a formal wedding",
 "Men's work attire for a corporate office environment",
 "Women's work attire for a corporate office environment",
 "Men's casual outfit for weekend brunch with friends"]

In [ ]:
res = []
errored_prompts = []
for prompt in all_prompts:
  # get 5 outfit recs for each prompt
  for i in range(5):
    try:
      my_wardrobe_subset = my_wardrobe.sample(60)
      clothing_items = [clean_json_string(s) for s in my_wardrobe_subset["json_string"]]
      sys_prompt = create_outfit_recommendation_prompt(clothing_items)
      output_dict = rec_pipeline(my_wardrobe_subset,sys_prompt, prompt)
      res.append(output_dict)
    except:
      print(prompt)
      errored_prompts.append(prompt)
      continue

Men's outfit for a formal wedding
Men's casual outfit for weekend brunch with friends
Men's athletic wear for a morning jog in the park
Men's cocktail attire for an evening charity gala
Women's cocktail attire for an evening charity gala
Men's minimalist outfit featuring neutral tones
Women's vintage-inspired look with earth tone color palette
Women's sophisticated monochromatic black outfit
Men's bohemian outfit with layered textures and patterns
Men's professional outfit in navy and gray color scheme
Men's all-white outfit for summer events
Men's all-white outfit for summer events
Men's all-white outfit for summer events
Men's all-white outfit for summer events
Men's all-white outfit for summer events
Women's all-white outfit for summer events
Women's all-white outfit for summer events
Women's all-white outfit for summer events
Women's edgy outfit combining black with red accent pieces
Men's outfit featuring complementary green and purple colors
Women's outfit featuring complementary

In [ ]:
errored_res = []
for prompt in errored_prompts:
  try:
    my_wardrobe_subset = my_wardrobe.sample(60)
    clothing_items = [clean_json_string(s) for s in my_wardrobe_subset["json_string"]]
    sys_prompt = create_outfit_recommendation_prompt(clothing_items)
    output_dict = rec_pipeline(my_wardrobe_subset,sys_prompt, prompt)
    errored_res.append(output_dict)
  except:
    print(prompt)

In [ ]:
res

In [ ]:
import json
with open('/content/drive/MyDrive/MIDS Capstone/recommendation_evals_2.json', 'w') as f:
    json.dump(res, f)

In [ ]:
with open('/content/drive/MyDrive/MIDS Capstone/recommendation_evals_errored_2.json', 'w') as f:
    json.dump(errored_res, f)